In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from scipy.ndimage import generic_filter
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

# ======== MODEL FUNCTIONS ========
def build_srcnn_model():
    input_img = Input(shape=(None, None, 1))
    x = Conv2D(64, (9, 9), padding='same', activation='relu')(input_img)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    output = Conv2D(1, (5, 5), padding='same', activation='linear')(x)
    model = Model(input_img, output)
    model.compile(optimizer='adam', loss='mse')
    return model

def build_classification_model(input_shape=(256, 256, 1)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(5, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ======== IMAGE PROCESSING ========
def load_and_process_slices(folder_path, num_slices, target_size=(256, 256), enhance=True):
    processed_slices = []
    if not os.path.exists(folder_path):
        print(f" Folder path does not exist: {folder_path}")
        return processed_slices

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp', '.dcm'))]
    image_files.sort()
    if len(image_files) == 0:
        print(f" No valid image files found in: {folder_path}")
        return processed_slices
    if len(image_files) != num_slices:
        print(f" Number of images in folder ({len(image_files)}) doesn't match metadata num_slices ({num_slices}). Using available images.")

    print(f" Loading {len(image_files[:num_slices])} images from {folder_path}...")
    for i, image_file in enumerate(tqdm(image_files[:num_slices])):
        try:
            img_path = os.path.join(folder_path, image_file)
            if image_file.lower().endswith('.dcm'):
                try:
                    import pydicom
                    ds = pydicom.dcmread(img_path)
                    img = ds.pixel_array
                    img = img.astype(np.float32)
                    img = (img - img.min()) / (img.max() - img.min())
                except ImportError:
                    print(" pydicom not installed. Cannot read DICOM files.")
                    continue
                except Exception as e:
                    print(f" Error reading DICOM file {image_file}: {e}")
                    continue
            else:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    try:
                        pil_img = Image.open(img_path).convert('L')
                        img = np.array(pil_img)
                    except Exception as e:
                        print(f" Failed to load image with both OpenCV and PIL: {img_path}")
                        continue

            if img is None:
                print(f" Failed to load image: {img_path}")
                continue

            img = cv2.resize(img, target_size)
            img = img.astype(np.float32) / 255.0
            if enhance:
                img = enhance_contrast(img)
            processed_slices.append(img)
        except Exception as e:
            print(f" Error processing image {image_file}: {e}")
    print(f" Successfully processed {len(processed_slices)} slices from {folder_path}")
    return processed_slices

def enhance_contrast(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply((image * 255).astype(np.uint8))
    return enhanced.astype(np.float32) / 255.0

def enhance_with_srcnn(slices, srcnn_model):
    print("Enhancing images with SRCNN...")
    enhanced_slices = []
    for slice in tqdm(slices):
        slice_expanded = np.expand_dims(slice, axis=0)
        slice_expanded = np.expand_dims(slice_expanded, axis=-1)
        enhanced = srcnn_model.predict(slice_expanded, verbose=0)
        enhanced = np.squeeze(enhanced)
        enhanced_slices.append(enhanced)
    return enhanced_slices

# ======== FEATURE EXTRACTION ========
def calculate_entropy(image):
    image = np.clip(image, 0, 1)
    hist = np.histogram(image, bins=256, range=(0, 1))[0]
    hist = hist / hist.sum()
    entropy = -np.sum(hist * np.log2(hist + 1e-10))
    return entropy

def calculate_homogeneity(image):
    try:
        std_dev = generic_filter(image, np.std, size=5)
        homogeneity = 1 / (1 + np.mean(std_dev))
        return homogeneity
    except:
        return 0.5

def calculate_symmetry(image):
    height, width = image.shape
    left_half = image[:, :width//2]
    right_half = image[:, width//2:]
    right_flipped = np.fliplr(right_half)
    if left_half.shape == right_flipped.shape:
        mse = np.mean((left_half - right_flipped) ** 2)
        symmetry = 1 / (1 + mse)
        return symmetry
    return 0.5

def estimate_cardiac_area(image):
    try:
        img_8bit = (image * 255).astype(np.uint8)
        _, thresh = cv2.threshold(img_8bit, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(largest_contour)
            return area / (image.shape[0] * image.shape[1])
    except Exception as e:
        print(f" Error estimating cardiac area: {e}")
    return 0

def extract_image_features(slices):
    print(" Extracting image features...")
    features = []
    for i, slice in enumerate(tqdm(slices)):
        try:
            slice_features = {
                'slice_number': i + 1,
                'mean_intensity': np.mean(slice),
                'std_intensity': np.std(slice),
                'max_intensity': np.max(slice),
                'min_intensity': np.min(slice),
                'contrast': np.max(slice) - np.min(slice),
                'entropy': calculate_entropy(slice),
                'homogeneity': calculate_homogeneity(slice),
                'cardiac_area': estimate_cardiac_area(slice),
                'symmetry_score': calculate_symmetry(slice)
            }
            features.append(slice_features)
        except Exception as e:
            print(f" Error extracting features from slice {i+1}: {e}")
            continue
    return features

# ======== SAVE COMPARISON ========
def save_comparison_images(patient_id, modality, slices, enhanced_slices):
    output_dir = os.path.join("output", f"comparison_{patient_id}", modality)
    os.makedirs(output_dir, exist_ok=True)
    for idx, (orig, enh) in enumerate(zip(slices, enhanced_slices), start=1):
        cv2.imwrite(os.path.join(output_dir, f"slice_{idx}_original.png"), (orig * 255).astype(np.uint8))
        cv2.imwrite(os.path.join(output_dir, f"slice_{idx}_enhanced.png"), (enh * 255).astype(np.uint8))

# ======== MAIN ========
if __name__ == "__main__":
    print("Loading dataset...")
    df = pd.read_csv('master_metadata.csv')

    # Get all unique patients
    patients = df['Patient_ID'].unique()

    # ✅ Limit to only 30 patients
    patients = patients[:30]

    print(f"Processing {len(patients)} patients (limited to 30)...")

    # Build models
    srcnn_model = build_srcnn_model()
    classification_model = build_classification_model()

    for patient_id in tqdm(patients, desc="Processing patients"):
        patient_data = df[df['Patient_ID'] == patient_id]
        results = {}
        if not patient_data.empty:
            for _, row in patient_data.iterrows():
                modality = row['Modality']
                folder_path = row['Folder_Path']
                num_slices = row['Num_Slices']
                age = row['Age']
                gender = row['Gender']

                slices = load_and_process_slices(folder_path, num_slices, enhance=True)
                if len(slices) == 0:
                    continue
                enhanced_slices = enhance_with_srcnn(slices, srcnn_model)
                image_features = extract_image_features(enhanced_slices)

                results[modality] = {
                    'slices': slices,
                    'enhanced_slices': enhanced_slices,
                    'image_features': image_features,
                    'patient_info': {
                        'age': age,
                        'gender': gender,
                        'modality': modality,
                        'num_slices_processed': len(slices)
                    }
                }

                save_comparison_images(patient_id, modality, slices, enhanced_slices)

            if results:
                print(f"\nSummary for patient {patient_id}:")
                for modality, data in results.items():
                    print(f"Modality: {modality}, Slices processed: {data['patient_info']['num_slices_processed']}")


Loading dataset...
Processing 30 patients (limited to 30)...


Processing patients:   0%|          | 0/30 [00:00<?, ?it/s]

 Loading 66 images from datasets/synthetic_slices/Patient_001/CT/...



100%|██████████| 66/66 [00:00<00:00, 356.69it/s]


 Successfully processed 66 slices from datasets/synthetic_slices/Patient_001/CT/
Enhancing images with SRCNN...


...

 Successfully processed 137 slices from datasets/synthetic_slices/Patient_030/MRI/
Enhancing images with SRCNN...



100%|██████████| 137/137 [00:23<00:00,  5.94it/s]


 Extracting image features...



Processing patients: 100%|██████████| 30/30 [2:51:59<00:00, 343.99s/it]


Summary for patient Patient_030:
Modality: CT, Slices processed: 125
Modality: MRI, Slices processed: 137


In [ ]:
import os
import zipfile
import shutil

def zip_output_folder(output_root="output", zip_name="Enhanced_Results.zip"):
    """
    Compresses all processed comparison folders into a single ZIP file.
    Example: output/comparison_1/, output/comparison_2/, etc.
    """
    if not os.path.exists(output_root):
        print(f"❌ Output folder not found: {output_root}")
        return None

    zip_path = os.path.join(os.getcwd(), zip_name)

    # If zip already exists, delete old version
    if os.path.exists(zip_path):
        os.remove(zip_path)

    print(f"📦 Creating ZIP file: {zip_name} ...")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(output_root):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=output_root)
                zipf.write(file_path, arcname)

    print(f"✅ ZIP file created successfully: {zip_path}")
    return zip_path

# Example Usage
if __name__ == "__main__":
    zip_path = zip_output_folder("output", "Enhanced_Comparison_Results.zip")

    # (Optional) Move ZIP into a separate folder
    dest_dir = "zipped_results"
    os.makedirs(dest_dir, exist_ok=True)
    if zip_path:
        shutil.move(zip_path, os.path.join(dest_dir, os.path.basename(zip_path)))
        print(f"📁 Moved ZIP to {dest_dir}/")


📦 Creating ZIP file: Enhanced_Comparison_Results.zip ...
✅ ZIP file created successfully: /content/Enhanced_Comparison_Results.zip
📁 Moved ZIP to zipped_results/
